# Text Analysis on germanwings data

In [19]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\germanwings.csv', encoding = 'utf-8', header = None)
df.columns = ['src_tweet_id', 'src_user_id', 'src_tweet','src_date','reply_tweet_id','reply_user_id','reply_tweet','reply_date', 'label']
df.reply_tweet = df.reply_tweet.replace(np.nan,"")

In [21]:
src_tw_df = df.iloc[:,[0,1,2,3,8]].copy()
src_tw_df = src_tw_df.drop_duplicates()
src_tw_df

,src_tweet_id,src_user_id,src_tweet,src_date,label
0,580319983676313601,8330472,#Últimahora Es tracta d'un aparell de Germanwi...,Tue Mar 24 10:47:20 +0000 2015,0
3,580321203757387776,92771309,Route map now being shared by http://t.co/LZkO...,Tue Mar 24 10:52:11 +0000 2015,0
4,580322026029654018,366089337,Pray for #4U9525 http://t.co/II7Rl24ffH,Tue Mar 24 10:55:27 +0000 2015,0
5,580322102307373056,16211566,Airbus A320 #4U9525 crash: \nFlight tracking d...,Tue Mar 24 10:55:45 +0000 2015,0
8,580322332348190720,92771309,Flightradar24 has this as the plane's last pos...,Tue Mar 24 10:56:40 +0000 2015,0
...,...,...,...,...,...
3916,581460720900050945,76453835,#Germanwings #CoPilot was a recent Muslim conv...,Fri Mar 27 14:20:13 +0000 2015,1
3938,581473088249958400,177584156,GERMAN NEWS REPORT: Co-Pilot of Germanwings Ai...,Fri Mar 27 15:09:22 +0000 2015,1
3945,581479017770979329,36116351,If the #Germanwings copilot #Lubitz was Muslim...,Fri Mar 27 15:32:55 +0000 2015,1
3965,581546828954411008,112874216,BREAKING: German Media Site Says Germanwings C...,Fri Mar 27 20:02:23 +0000 2015,1


## Prepocessing
- Define function to process text at one time

In [22]:
def textprocessing(text):
    text = text.lower()

    text = " ".join([word for word in text.split() if 'http' not in word
                                and not word.startswith('@')])
                                #and word != 'RT'])
    import re
    def remove_punct(tweet):
        new_words = []
        for word in tweet:
            w = re.sub(r'[^\w\s]','',word) #remove everything except words and space
            w = re.sub(r'_','',w) #how to remove underscore as well
            new_words.append(w)

        return new_words
    text = "".join(remove_punct(text))
          
    from nltk.tokenize import TweetTokenizer
    tknzr = TweetTokenizer(strip_handles=True)
    text = tknzr.tokenize(text)
    
    from nltk.stem.porter import PorterStemmer
    stem = PorterStemmer()
    text = [stem.stem(i) for i in text]
    
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words("english"))
    def filterstopwords(tw):
        filter_stopwords = []
        for w in tw:
            if w not in stop_words:
                filter_stopwords.append(w)
        return filter_stopwords
    text = filterstopwords(text)
    text = " ".join(text)
    
    return text

In [23]:
print("original text: ", df.reply_tweet.iloc[20])
print("cleaned text: ", textprocessing(df.reply_tweet.iloc[20]))

original text:  @Youngloch yep. That's what I was thinking.
cleaned text:  yep wa think


### applied on original data (**df**)

In [24]:
df['cleaned_src_tw'] = df.src_tweet.apply(textprocessing)
df['cleaned_reply_tw'] = df.reply_tweet.apply(textprocessing)

In [25]:
df.head()

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label,cleaned_src_tw,cleaned_reply_tw
0,580319983676313601,8330472,#Últimahora Es tracta d'un aparell de Germanwi...,Tue Mar 24 10:47:20 +0000 2015,580322851850461184,2307392966,“@324cat: #Últimahora Es tracta d'un aparell d...,Tue Mar 24 10:58:44 +0000 2015,0,últimahora es tracta dun aparel de germanw amb...,324cat últimahora es tracta dun aparel de germ...
1,580319983676313601,8330472,#Últimahora Es tracta d'un aparell de Germanwi...,Tue Mar 24 10:47:20 +0000 2015,580323127089082368,2535310842,@senyera @324cat 148 pasajeros 142+6,Tue Mar 24 10:59:50 +0000 2015,0,últimahora es tracta dun aparel de germanw amb...,148 pasajero 1426
2,580319983676313601,8330472,#Últimahora Es tracta d'un aparell de Germanwi...,Tue Mar 24 10:47:20 +0000 2015,580325737619685377,2535310842,@senyera @324cat dice el ministro frances que ...,Tue Mar 24 11:10:12 +0000 2015,0,últimahora es tracta dun aparel de germanw amb...,dice el ministro franc que hay supervivient
3,580321203757387776,92771309,Route map now being shared by http://t.co/LZkO...,Tue Mar 24 10:52:11 +0000 2015,580324361342615552,161546098,結構な山の中だぞ、それは…… “@BBCRosAtkins: Route map now b...,Tue Mar 24 11:04:44 +0000 2015,0,rout map share 4u9525,結構な山の中だぞそれは bbcrosatkin rout map share 4u9525
4,580322026029654018,366089337,Pray for #4U9525 http://t.co/II7Rl24ffH,Tue Mar 24 10:55:27 +0000 2015,580332543087505408,744921588,“@f4izalhassan: Pray for #4U9525 http://t.co/d...,Tue Mar 24 11:37:14 +0000 2015,0,pray 4u9525,f4izalhassan pray 4u9525 إنا لله وإنـا إليه را...


### applied on source tweet data (**src_tw_df**) 

In [26]:
src_tw_df['cleaned_src_tw'] = src_tw_df.src_tweet.apply(textprocessing)

## Sentiment Analysis

In [27]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [28]:
def get_sentiment(tweet):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(tweet)
    if vs['compound'] >=0.05:
        sentiment_label = 'Positive'
    elif (vs['compound'] > -0.05) & (vs['compound'] < 0.05):
        sentiment_label = 'Neutral'
    elif vs['compound']<= -0.05:
        sentiment_label = 'Negative' 
    result = sentiment_label
    return result

### applied on original df

In [29]:
df['src_sentiment']= df.cleaned_src_tw.apply(get_sentiment)
df['reply_sentiment']= df.cleaned_reply_tw.apply(get_sentiment)

In [30]:
df.head(1)

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label,cleaned_src_tw,cleaned_reply_tw,src_sentiment,reply_sentiment
0,580319983676313601,8330472,#Últimahora Es tracta d'un aparell de Germanwi...,Tue Mar 24 10:47:20 +0000 2015,580322851850461184,2307392966,“@324cat: #Últimahora Es tracta d'un aparell d...,Tue Mar 24 10:58:44 +0000 2015,0,últimahora es tracta dun aparel de germanw amb...,324cat últimahora es tracta dun aparel de germ...,Neutral,Neutral


### applied on source tweet data only 

In [31]:
### source tweet data only 
src_tw_df['src_sentiment']=  src_tw_df['cleaned_src_tw'].apply(get_sentiment)

### Visualization

In [ ]:
print(df[df.label ==1]['reply_sentiment'].value_counts())
plt.figure(figsize=(8,6))
sns.countplot(x='reply_sentiment',data= df[df['label'] == 1])

plt.suptitle('Sentiment analysis on Reply tweets under Catergory Rumour',fontsize=16)
plt.title('Reply tweets show more positive attitude towards rumour tweet',fontsize=12,color='grey')
plt.savefig('graph/germanwings/senti_reply_rumour.pdf',dpi=300)

In [ ]:
print(df[df.label ==0]['reply_sentiment'].value_counts())
plt.figure(figsize=(8,6))
sns.countplot(x='reply_sentiment',data= df[df['label'] == 0])

plt.suptitle('Sentiment analysis on Reply tweets under Catergory Non-Rumour',fontsize=16)
plt.title('Reply tweets show more neutrual attitude towards non-rumour tweet',fontsize=12,color='grey')
plt.savefig('graph/germanwings/senti_reply_nonrumour.pdf',dpi=300)

In [ ]:
print(src_tw_df['src_sentiment'].value_counts())
sns.countplot(x='src_sentiment',data =src_tw_df,hue='label')

plt.suptitle('Sentiment analysis on Source Tweets',fontsize=16)
plt.title('Rumour source tweets show more neutral attitude.',fontsize=12,color='grey')
plt.savefig('graph/germanwings/senti_source.pdf',dpi=300)

### save data to csv 

In [ ]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\germanwings-df.csv',index=False)
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\germanwings-src.csv',index=False)

## Keywords Extraction
### WordCloud

In [ ]:
from wordcloud import WordCloud

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(docx)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()

#### Reply Tweets

In [ ]:
#nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [ ]:
reply_sen = df.cleaned_reply_tw.apply(word_tokenize)
reply_tw_list = []
for sen in reply_sen:
    for token in sen:
        reply_tw_list.append(token)
reply_tw_doc = " ".join(reply_tw_list)

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(reply_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/germanwings/wordcloud_reply.pdf')

In [ ]:
# positive 
posi_reply_sen = df[df.reply_sentiment=='Positive'].cleaned_reply_tw.apply(word_tokenize)
posi_reply_tw_list = []
for sen in posi_reply_sen:
    for token in sen:
        posi_reply_tw_list.append(token)
posi_reply_tw_doc = " ".join(posi_reply_tw_list)


In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(posi_reply_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/germanwings/wordcloud_reply_posi.pdf')

#### Source Tweet

In [ ]:
src_sen = src_tw_df.cleaned_src_tw.apply(word_tokenize)
src_tw_list = []
for sen in src_sen:
    for token in sen:
        src_tw_list.append(token)

src_tw_doc = " ".join(src_tw_list)

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(src_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/germanwings/wordcloud_src.pdf')

### Common

In [ ]:
from collections import Counter

In [ ]:
def get_tokens(docx,num=30):
    word_tokens = Counter(docx)
    most_common = word_tokens.most_common(num)
    result = dict(most_common)
    return result

In [ ]:
pd.DataFrame(get_tokens(reply_tw_list).items(),columns=['word','freq']).plot(kind='bar',x='word',y='freq')
plt.title("Most frequent words among reply tweets are 'Prince','show','tonight'.")

In [ ]:
pd.DataFrame(get_tokens(src_tw_list).items(),columns=['word','freq']).plot(kind='bar',x='word',y='freq')
plt.title("Most frequent words among source tweets are 'Price', 'show', 'tonight'")


## Emotion Detection

### text2emotion package
https://snyk.io/advisor/python/text2emotion 

In [32]:
import text2emotion as te

In [33]:
emotion = df.cleaned_reply_tw.apply(te.get_emotion)

In [38]:
emo = emotion.apply(lambda x: max(x,key=x.get))
df['reply_emotion']=emo

In [39]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\germanwings-df.csv',index=False)

In [40]:
emotion1 = src_tw_df.cleaned_src_tw.apply(te.get_emotion)
emo1 = emotion.apply(lambda x: max(x,key=x.get))
src_tw_df['src_emotion']=emo1

In [41]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\germanwings-src.csv',index=False)